# Importing and installing packages

In [1]:
import Pkg; Pkg.add("Revise")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [2]:
import Pkg; Pkg.add("PhyloTools")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [3]:
import Pkg; Pkg.add("TreeTools")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [4]:
import Pkg; Pkg.add("DCAUtils")
import Pkg; Pkg.add("JLD2")
import Pkg; Pkg.add("DataFrames")
import Pkg; Pkg.add("PyPlot")
import Pkg; Pkg.add("Statistics")
import Pkg; Pkg.add("DelimitedFiles")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [5]:
using JLD2

x  = 3;
y = zeros(3, 3);

@save "ciao.jld2" x y 

In [6]:
@load "../ciao.jld2" 

2-element Vector{Symbol}:
 :x
 :y

In [7]:
using DCAUtils

msa = read_fasta_alignment("../PF72/PF00072_collapsed.fasta", 0.9);

In [8]:
msa[:,1]

112-element Vector{Int8}:
 21
 10
 10
 18
  4
  3
  3
  4
 17
 10
  ⋮
  8
  6
 17
 10
 10
 17
  9
 10
 21

## Getting sequences with biggest and lowest CDE, and something in the middle

## Launching the algorithm

In [10]:
root = msa[:,1]

112-element Vector{Int8}:
 21
 10
 10
 18
  4
  3
  3
  4
 17
 10
  ⋮
  8
  6
 17
 10
 10
 17
  9
 10
 21

## Function to read bm parameters

In [11]:
using Revise, PhyloTools, TreeTools, DCAUtils, JLD2, PyPlot, Statistics, DelimitedFiles


In [23]:
import Pkg; Pkg.add(url="https://github.com/matteobisardi/KitMSA.git")

     Cloning git-repo `https://github.com/matteobisardi/KitMSA.git`
    Updating git-repo `https://github.com/matteobisardi/KitMSA.git`
   Resolving package versions...
   Installed Arpack ──────────── v0.5.4
   Installed Arpack_jll ──────── v3.5.1+1
   Installed Distances ───────── v0.10.12
   Installed MultivariateStats ─ v0.10.3
    Updating `~/.julia/environments/v1.11/Project.toml`
  [e1e610c1] + KitMSA v0.1.0 `https://github.com/matteobisardi/KitMSA.git#main`
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [7d9fca2a] + Arpack v0.5.4
  [b4f34e82] + Distances v0.10.12
  [e1e610c1] + KitMSA v0.1.0 `https://github.com/matteobisardi/KitMSA.git#main`
  [6f286f6a] + MultivariateStats v0.10.3
⌅ [68821587] + Arpack_jll v3.5.1+1
        Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated -m`
Precompiling project...
    750.8 ms  ✓ Arpack_jll
   1395.2 ms  ✓ Distances
    662.6 ms  ✓

In [24]:
import KitMSA: letter2num, num2letter

In [25]:
function read_par_BM_0gapsave(path::AbstractString, q::Integer = 21)
    params = readdlm(path,' ', use_mmap = true)[:, 2:6]
    l_file = size(params, 1)
    N = Integer(((q - 2) + sqrt( (q-2)^2 + 8*l_file))/(2*q))
    J = Array{Float64}(undef, q, q, N, N)
    h = Array{Float64}(undef, q, N)
    n_J = Int(q*q*N*(N-1)/2)
    n_h = q*N
    for k in 1:n_J
        i, j, a, b, par_j = params[k, :]
        i += 1
        j += 1
        a == 0 && (a = q)
        b == 0 && (b = q)
        J[a, b, i, j] = par_j
    end
    for l in (n_J + 1): n_h + n_J
        i, a, par_h = params[l, :]
        i += 1
        a == 0 && (a = q)
        h[a, i] = par_h
    end
    return h, J
end
### when in the file aminoacids are indicated as -,A,C,...
function read_par_BM_lettersave(path::AbstractString, q::Integer = 21)
    params = readdlm(path,' ', use_mmap = true)[:, 2:6]
    l_file = size(params, 1);
    N = Integer(((q - 2) + sqrt( (q-2)^2 + 8*l_file))/(2*q));
    J = Array{Float64}(undef, q, q, N, N);
    h = Array{Float64}(undef, q, N);
    n_J = Int(q*q*N*(N-1)/2);
    n_h = q*N;
    for k in 1:n_J
        i, j, a, b, par_j = params[k, :]
        i += 1
        j += 1
        J[letter2num(a[1]), letter2num(b[1]), i, j] = par_j
    end
    for l in (n_J + 1): n_h + n_J
        i, a, par_h = params[l, :]
        i += 1
        h[letter2num(a[1]), i] = par_h
    end
    return h, J
end
function symmetrize_Jsave(J_old::Array{Float64, 4})
   J_s = deepcopy(J_old)
   q,q, N, N = size(J_old)
   for i in 1:N
        for j in i+1:N
            for a in 1:q
                for b in 1:q
                      J_s[b, a, j, i]  = J_old[a, b, i, j]
                end
            end
        end
    end
    return J_s
end
function set_max_field_to_0save(h_old::Array{Float64, 2}, q::Int = 21)
   h_new = deepcopy(h_old)
   q, N = size(h_old)
   for i in 1:N
        hmax = maximum([h_old[a, i] for a in 1:q])
        h_new[:, i] .-= hmax
   end
   return h_new
end

function read_par_BM_lettersave(path::AbstractString, q::Integer = 21)
    params = readdlm(path,' ', use_mmap = true)[:, 2:6]
    l_file = size(params, 1);
    N = Integer(((q - 2) + sqrt( (q-2)^2 + 8*l_file))/(2*q));
    J = Array{Float64}(undef, q, q, N, N);
    h = Array{Float64}(undef, q, N);
    n_J = Int(q*q*N*(N-1)/2);
    n_h = q*N;
    for k in 1:n_J
        i, j, a, b, par_j = params[k, :]
        i += 1
        j += 1
        J[letter2num(a[1]), letter2num(b[1]), i, j] = par_j
    end
    for l in (n_J + 1): n_h + n_J
        i, a, par_h = params[l, :]
        i += 1
        h[letter2num(a[1]), i] = par_h
    end
    return h, J
end

read_par_BM_lettersave (generic function with 2 methods)

In [26]:
# Read the parameters

h_tmp, J_tmp = read_par_BM_lettersave("../PF72/PF72_params.dat");
h = set_max_field_to_0save(h_tmp);
J_tmp2 = symmetrize_Jsave(J_tmp);
J = permutedims(J_tmp2, [1,3,2,4])    #questo è come leggo i parametri

21×112×21×112 Array{Float64, 4}:
[:, :, 1, 1] =
 0.0   9.52909e-7   9.56576e-7   1.03749e-6  …   9.38837e-7   9.65354e-7
 0.0   9.62898e-7   9.43171e-7   9.36789e-7      9.63493e-7   9.65354e-7
 0.0   9.65354e-7   9.65354e-7   9.6488e-7       9.65354e-7   9.65354e-7
 0.0   9.65906e-7   9.65354e-7   9.65354e-7      9.65354e-7   9.5182e-7
 0.0   9.50271e-7   9.59772e-7   9.62303e-7      9.62378e-7   9.65354e-7
 0.0   9.6143e-7    9.65906e-7   9.66987e-7  …   9.65354e-7   9.65354e-7
 0.0   9.60965e-7   9.58608e-7   9.65354e-7      9.65354e-7   9.65854e-7
 0.0   1.06411e-6  -1.41385e-5   9.81659e-7     -4.23919e-6   9.65354e-7
 0.0   9.65354e-7   9.65354e-7   9.65354e-7      9.65354e-7  -3.80698e-6
 0.0  -1.94611e-5  -3.98854e-6   9.17313e-7      8.89608e-7   9.65354e-7
 ⋮                                           ⋱   ⋮           
 0.0   9.65354e-7   9.65354e-7   9.65354e-7      9.65354e-7   9.65354e-7
 0.0   9.66163e-7   9.66699e-7   9.65354e-7      9.65354e-7   9.53307e-7
 0.0   9.6411e-

In [28]:
size(h,1), size(J)

(21, (21, 112, 21, 112))

In [37]:
Base.@kwdef mutable struct Seq <: TreeNodeData # Create a custom data type
    seq::Array{Int,1} = [0,0]
    DNA::Array{String,1} = ["A","b"]
end


Seq

In [ ]:
using Revise, PhyloTools, TreeTools, DCAUtils, JLD2, PyPlot, Statistics, DelimitedFiles


nat_msa  = read_fasta_alignment("PF72/PF00072_collapsed.fasta", 0.9);
w = compute_weights(nat_msa, 22, 0.2)[1];
#@load ""; h = h_dbd; J = J_dbd; # read them with Leo's function first
#@load "../data_Anc/3_start_seq_and_sweeps4DBD_ASR.jld2"

q = 21; L = 112;

#tree_file = "../data_Anc/DBD_tree_midpoint_rooted.nwk"; 
#tree_file = "../data_Anc/DBD_tree_clean_short_300_rooted_midpoint"
#tree_file = "../data_Anc/DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301.nwk"
tree_file = "PF72/PF00072_tree_collapsed_noonlychild_midpointrooted.nwk"
tree = read_tree(tree_file, node_data_type = Seq);
dd = [TreeTools.distance(tree.root, a) for a in leaves(tree)]; # Root to leaf distances

branch_d = mean([TreeTools.distance(tree.root, a) for a in leaves(tree)]); # Mean root-leaf distance
sweeps = [3, 6, 30, 60] # MCMC sweeps to be used for the evolution
mus = sweeps ./ branch_d; 

folder = "PF72/generated_leaves/"

if !isdir(folder)
    mkpath(folder)
end

# Define three starting sequences
start_msa = nat_msa[:, 1:3]; # Do it with CDE

# step_msa is an array of MSAs for each sweep
step_msa_1 = [];step_msa_2 = [];step_msa_3 = []; res_all1 = []; res_all2 = []; res_all3 = [];
for mu in mus 
    @time res1 = PhyloTools.run_evolution_ontree(Int.(start_msa[:,1]), tree_file, 
    h, J, mu = mu, p = 0.5); 
    push!(res_all1, res1)
    push!(step_msa_1, msa_from_leafs(res1))
    @time res2 = PhyloTools.run_evolution_ontree(Int.(start_msa[:,2]), tree_file, 
    h, J, mu = mu, p = 0.5); 
    push!(res_all2, res2)
    push!(step_msa_2, msa_from_leafs(res2))
    @time res3 = PhyloTools.run_evolution_ontree(Int.(start_msa[:,3]), tree_file, 
    h, J, mu = mu, p = 0.5); 
    push!(res_all3, res3)
    push!(step_msa_3, msa_from_leafs(res3))
end

for i in 1:length(mus)
    leavestofasta(joinpath(folder, "seq1_mu$(round(mus[i], 
        digits = 2)).fa"), res_all1[i])
    leavestofasta(joinpath(folder, "seq2_mu$(round(mus[i], 
        digits = 2)).fa"), res_all2[i])
    leavestofasta(joinpath(folder, "seq3_mu$(round(mus[i], 
        digits = 2)).fa"), res_all3[i])
end



θ = 0.2 threshold = 22.0
M = 49184 N = 112 Meff = 43555.058320922915


UndefVarError: UndefVarError: `run_evolution_ontree` not defined in `PhyloTools`
Suggestion: check for spelling errors or missing imports.

In [43]:
PhyloTools.run_evolution_ontree(start_msa, tree_file, h, J, mu = 0.1, p = 0.5)

UndefVarError: UndefVarError: `run_evolution_ontree` not defined in `PhyloTools`
Suggestion: check for spelling errors or missing imports.

In [34]:
PhyloTools.Seq

UndefVarError: UndefVarError: `Seq` not defined in `PhyloTools`
Suggestion: check for spelling errors or missing imports.

In [ ]:


# Analysis of the generated data

ds = [pairwise_ham_dist(Int8.(step_msa_3[i]), all = true) for i in 1:5];
close("all"); 
for i in 1:5
    plt.hist(ds[i], label = "t $(i)"); 
end
plt.savefig("../prova_distanze.png")
            
@save "PF72/generated_leaves/" step_msa_1 step_msa_2 step_msa_3 

d1 = [mean(ham_dist(start_msa[:,1], Int8.(x))) for x in step_msa_1] ./L ;
d2 = [mean(ham_dist(start_msa[:,2], Int8.(x))) for x in step_msa_2] ./L ;
d3 = [mean(ham_dist(start_msa[:,3], Int8.(x))) for x in step_msa_3] ./L ;
chi1 = [var(ham_dist(start_msa[:,1], Int8.(x))) for x in step_msa_1] ./(L^2) ;
chi2 = [var(ham_dist(start_msa[:,2], Int8.(x))) for x in step_msa_2] ./(L^2) ;
chi3 = [var(ham_dist(start_msa[:,3], Int8.(x))) for x in step_msa_3] ./(L^2) ;

ts = deepcopy(sweeps[[1,2,3,4,6]]);
close("all"); plt.scatter(ts, d1, color = "blue"); plt.scatter(ts, 
    d2, color = "red"); plt.scatter(ts, d3, color = "green");plt.ylabel("Hamming Distance");plt.xlabel("MCMC Sweeps"); plt.xscale("log"); savefig("../ciao_d.png")

close("all"); plt.scatter(ts, chi1, color = "blue"); plt.scatter(ts, 
    chi2, color = "red"); plt.scatter(ts, chi3, color = "green"); plt.xlabel("MCMC Sweeps");plt.ylabel("Chi_dyn^A"); plt.xscale("log"); savefig("../ciao_chi.png")




inf_mus1 = [];inf_mus2 = [];inf_mus3 = [];
for mu in mus 
    push!(inf_mus1, infer_mu(tree_file, joinpath(folder, "seq1/seq1_mu$(round(mu, 
               digits = 2)).fa")))
    push!(inf_mus2, infer_mu(tree_file, joinpath(folder, "seq2/seq2_mu$(round(mu, 
               digits = 2)).fa")))
    push!(inf_mus3, infer_mu(tree_file, joinpath(folder, "seq3/seq3_mu$(round(mu, 
               digits = 2)).fa")))
end

writedlm(joinpath(folder, "seq1/inferred_mus1.txt"), inf_mus1)
writedlm(joinpath(folder, "seq2/inferred_mus2.txt"), inf_mus2)
writedlm(joinpath(folder, "seq3/inferred_mus3.txt"), inf_mus3)



#analysis on generated data

using Revise, PhyloTools, TreeTools, DCAUtils, JLD2, PyPlot, Statistics, DelimitedFiles


nat_msa  = read_fasta_alignment("../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m", 0.9);
w = compute_weights(nat_msa, 22, 0.2)[1];
@load "../data_Genie/pars_dbd.jld2"; h = h_dbd; J = J_dbd;
@load "../data_Anc/3_start_seq_and_sweeps4DBD_ASR.jld2"

q = 21; L =76;

#tree_file = "../data_Anc/DBD_tree_midpoint_rooted.nwk"; 
#tree_file = "../data_Anc/DBD_tree_clean_short_300_rooted_midpoint"
tree_file = "../data_Anc/DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301.nwk"
tree = read_tree(tree_file, node_data_type = Seq);
dd = [distance(tree.root, a) for a in leaves(tree)];

branch_d = mean([distance(tree.root, a) for a in leaves(tree)]);
#mus = sweeps[[1,2,3,4,6]] ./ branch_d;
mus = readdlm("../data_Anc/OFF_3_seq_DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301/mus.txt")[:]
sweeps = mus .* branch_d;

folder = "../data_Anc/OFF_3_seq_DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301/"


res_ASR = zeros(Int, 3, L, length(mus));
pp = Matrix{Matrix{Float64}}(undef, 3, length(mus));
rec_msas = Matrix{Matrix{Int}}(undef, 3, length(mus));
ens = Matrix{Array{Float64,1}}(undef, 3, length(mus));
hams = Matrix{Array{Float64,1}}(undef, 3, length(mus));
rec_msas_reshuf = Matrix{Matrix{Int}}(undef, 3, length(mus));
ens_reshuf = Matrix{Array{Float64,1}}(undef, 3, length(mus));
hams_reshuf = Matrix{Array{Float64,1}}(undef, 3, length(mus));
ens_ML =  zeros(3, length(mus));
hams_ML =  zeros(3, length(mus));
step_msa_1 = [];step_msa_2 = [];step_msa_3 = [];
step_msa = [step_msa_1, step_msa_2, step_msa_3];

for i in 1:3
    inf_mus = readdlm(joinpath(folder, "seq$(i)/inferred_mus$(i).txt"))
    println(inf_mus)
    for n in 1:length(mus)
        file_seqs = joinpath(folder, "seq$(i)/seq$(i)_mu$(round(mus[n], 
        digits = 2)).fa")
        push!(step_msa[i],read_fasta_alignment(file_seqs,0.9))
        res, p = Felsenstein2(file_seqs, tree_file, file_seqs, inf_mus[n])
        res_ASR[i,:,n] .= Int.(res)
        pp[i,n] = p
        rec_msas[i,n] = sampling_ANC(pp[i,n], n_seq = 500)
        hams[i,n] = ham_dist(Int.(start_msa[:,i]), rec_msas[i,n])
        ens[i,n] = energy(rec_msas[i,n], h, J)
        #@time rec_msas_reshuf[i,n] = reshuffle_entr(rec_msas[i,n], h, J, temp = 0.2)
        #hams_reshuf[i,n] = ham_dist(Int.(start_msa[:,i]), rec_msas_reshuf[i,n])
        #ens_reshuf[i,n] = energy(rec_msas_reshuf[i,n], h, J)
        hams_ML[i,n] = ham_dist(Int.(start_msa[:,i]), res_ASR[i,:,n])
        ens_ML[i,n] = energy(res_ASR[i,:,n], h, J)
    end
end

close("all")
fig, axs = plt.subplots(3, length(mus), figsize = (20,12) )
for i in 1:3
    for n in 1:length(mus)
        axs[i,n].scatter(ens[i,n], hams[i,n], marker = "o", alpha = 0.5, color = "blue")
        axs[i,n].scatter(ens_reshuf[i,n], hams_reshuf[i,n], marker = "^", alpha = 0.5, color = "red")
        axs[i,n].scatter(ens_ML[i,n], hams_ML[i,n], marker = "x", s = 150, color = "black")
    end
end

for i in 1:3
    axs[i,1].set_ylabel("Seq $(i)", fontsize = 20)
end

for n in 1:length(mus)
    axs[1,n].set_title("Mu =  $(round(mus[n], digits = 3))", fontsize = 20) 
end
    
fig.supxlabel("Energy", fontsize = 30)
fig.supylabel("Hamming", fontsize = 30)
savefig("../ens_vs_ham_ASR_tot.png")


@save "../res_ASR.jld2" res_ASR pp rec_msas ens hams ens_ML hams_ML mus




@load "PF72/PF00072_tree_collapsed_noonlychild_midpointrooted.nwk"

        
H_ASR = zeros(3,length(mus));        
H_evol = zeros(3,length(mus));
for i in 1:3
    for n in 1:length(mus)
        H_ASR[i,n] = ham_dist(Int.(start_msa[:,i]), Int.(res_ASR[i,:,n]))
        H_evol[i,n] = mean(ham_dist(Int.(start_msa[:,i]), Int.(step_msa[i][n])))
    end 
end

cc = ["blue", "red", "green"];
close("all"); 
for i in 1:3
    plt.scatter(sweeps, H_ASR[i,:] ./L, marker = "*", label = "H_asr $(i)", color = cc[i]); 
    plt.scatter(sweeps, H_evol[i,:] ./L, marker = "o", label = "H_evol $(i)", color = cc[i]); 
    plt.xscale("log"); 
    plt.xlabel("Average root-leaf sweeps"); 
    plt.ylabel("Hamming distance");
    plt.legend(); 
end 
savefig("../prova_asr.png")



In [ ]:
using Revise, PhyloTools, TreeTools, DCAUtils, JLD2, PyPlot, Statistics, DelimitedFiles


nat_msa  = read_fasta_alignment("../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m", 0.9);
w = compute_weights(nat_msa, 22, 0.2)[1];
@load "../data_Genie/pars_dbd.jld2"; h = h_dbd; J = J_dbd;
@load "../data_Anc/3_start_seq_and_sweeps4DBD_ASR.jld2"

q = 21; L =76;

#tree_file = "../data_Anc/DBD_tree_midpoint_rooted.nwk"; 
#tree_file = "../data_Anc/DBD_tree_clean_short_300_rooted_midpoint"
tree_file = "../data_Anc/DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301.nwk"
tree = read_tree(tree_file, node_data_type = Seq);
dd = [distance(tree.root, a) for a in leaves(tree)];

branch_d = mean([distance(tree.root, a) for a in leaves(tree)]);
mus = sweeps[[1,2,3,4,6]] ./ branch_d;

folder = "../data_Anc/OFF_3_seq_DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301"

res_ASR = zeros(Int, 3, L, length(mus));
pp = Matrix{Matrix{Float64}}(undef, 3, length(mus));
rec_msas = Matrix{Matrix{Int}}(undef, 3, length(mus));
rec_msas_reshuf = Matrix{Matrix{Int}}(undef, 3, length(mus));
ens = Matrix{Array{Float64,1}}(undef, 3, length(mus));
hams = Matrix{Array{Float64,1}}(undef, 3, length(mus));
rec_msas_reshuf_lowT = Matrix{Matrix{Int}}(undef, 3, length(mus));
ens_reshuf = Matrix{Array{Float64,1}}(undef, 3, length(mus));
hams_reshuf = Matrix{Array{Float64,1}}(undef, 3, length(mus));
ens_reshuf_lowT = Matrix{Array{Float64,1}}(undef, 3, length(mus));
hams_reshuf_lowT = Matrix{Array{Float64,1}}(undef, 3, length(mus));
ens_ML =  zeros(3, length(mus));
hams_ML =  zeros(3, length(mus));

for i in 1:3
    inf_mus = readdlm(joinpath(folder, "seq$(i)/inferred_mus$(i).txt"))
    println(inf_mus)
    for n in 4:length(mus)
        file_seqs = joinpath(folder, "seq$(i)/seq$(i)_mu$(round(mus[n], 
        digits = 2)).fa")
        res, p = Felsenstein2(file_seqs, tree_file, file_seqs, inf_mus[n])
        res_ASR[i,:,n] .= Int.(res)
        pp[i,n] = p
        rec_msas[i,n] = sampling_ANC(pp[i,n], n_seq = 500)
        hams[i,n] = ham_dist(Int.(start_msa[:,i]), rec_msas[i,n])
        ens[i,n] = energy(rec_msas[i,n], h, J)
        @time rec_msas_reshuf[i,n] = reshuffle_entr(rec_msas[i,n], h, J, temp = 0.8)
        @time rec_msas_reshuf_lowT[i,n] = reshuffle_entr(rec_msas[i,n], h, J, temp = 0.2)
        hams_reshuf[i,n] = ham_dist(Int.(start_msa[:,i]), rec_msas_reshuf[i,n])
        ens_reshuf[i,n] = energy(rec_msas_reshuf[i,n], h, J)
        hams_reshuf_lowT[i,n] = ham_dist(Int.(start_msa[:,i]), rec_msas_reshuf_lowT[i,n])
        ens_reshuf_lowT[i,n] = energy(rec_msas_reshuf_lowT[i,n], h, J)
        hams_ML[i,n] = ham_dist(Int.(start_msa[:,i]), res_ASR[i,:,n])
        ens_ML[i,n] = energy(res_ASR[i,:,n], h, J)
    end
end

close("all")
fig, axs = plt.subplots(3, 2, figsize = (12,16) )
for i in 1:3
    for n in 4:length(mus)
        axs[i,n-3].scatter(ens[i,n], hams[i,n], marker = "o", alpha = 0.5, color = "blue")
        axs[i,n-3].scatter(ens_reshuf[i,n], hams_reshuf[i,n], marker = "^", alpha = 0.5, color = "red")
        axs[i,n-3].scatter(ens_reshuf_lowT[i,n], hams_reshuf_lowT[i,n], marker = "*", alpha = 0.5, color = "green")
        axs[i,n-3].scatter(ens_ML[i,n], hams_ML[i,n], marker = "x", s = 150, color = "black")
    end
end

for i in 1:3
    axs[i,1].set_ylabel("Seq $(i)", fontsize = 20)
end

for n in 4:length(mus)
    axs[1,n-3].set_title("Mu =  $(round(mus[n], digits = 3))", fontsize = 20) 
end
    
fig.supxlabel("Energy", fontsize = 30)
fig.supylabel("Hamming", fontsize = 30)
savefig("../ens_vs_ham_ASR_diff_temps.png")




In [ ]:
using Revise, PhyloTools, TreeTools, DCAUtils, JLD2, PyPlot, Statistics, DelimitedFiles

nat_msa  = read_fasta_alignment("../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m", 0.9);
w = compute_weights(nat_msa, 22, 0.2)[1];
@load "../data_Genie/pars_dbd.jld2"; h = h_dbd; J = J_dbd;
@load "../data_Anc/3_start_seq_and_sweeps4DBD_ASR.jld2"

q = 21; L =76;

tree_file = "../data_Anc/DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301.nwk"
tree = read_tree(tree_file, node_data_type = Seq);
dd = [distance(tree.root, a) for a in leaves(tree)];

branch_d = mean([distance(tree.root, a) for a in leaves(tree)]);
mus = sweeps[[1,2,3,4,6]] ./ branch_d;

folder = "../data_Anc/OFF_3_seq_DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301"

step_msa_1 = [];step_msa_2 = [];step_msa_3 = []; res_all1 = []; res_all2 = []; res_all3 = [];

            
@load "../data_Anc/DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301.jld2" step_msa_1 step_msa_2 step_msa_3 

d1 = [mean(ham_dist(start_msa[:,1], Int8.(x))) for x in step_msa_1] ./L ;
d2 = [mean(ham_dist(start_msa[:,2], Int8.(x))) for x in step_msa_2] ./L ;
d3 = [mean(ham_dist(start_msa[:,3], Int8.(x))) for x in step_msa_3] ./L ;
chi1 = [var(ham_dist(start_msa[:,1], Int8.(x))) for x in step_msa_1] ./(L^2) ;
chi2 = [var(ham_dist(start_msa[:,2], Int8.(x))) for x in step_msa_2] ./(L^2) ;
chi3 = [var(ham_dist(start_msa[:,3], Int8.(x))) for x in step_msa_3] ./(L^2) ;

ts = deepcopy(sweeps[[1,2,3,4,6]]);
close("all"); plt.scatter(ts, d1, color = "blue"); plt.scatter(ts, 
    d2, color = "red"); plt.scatter(ts, d3, color = "green");plt.ylabel("Hamming Distance");plt.xlabel("MCMC Sweeps"); plt.xscale("log"); savefig("../ciao_d.png")

close("all"); plt.scatter(ts, chi1, color = "blue"); plt.scatter(ts, 
    chi2, color = "red"); plt.scatter(ts, chi3, color = "green"); plt.xlabel("MCMC Sweeps");plt.ylabel("Chi_dyn^A"); plt.xscale("log"); savefig("../ciao_chi.png")




inf_mus1 = [];inf_mus2 = [];inf_mus3 = [];
for mu in mus 
    @time push!(inf_mus1, infer_mu(tree_file, joinpath(folder, "seq1/seq1_mu$(round(mu, 
               digits = 2)).fa")))
    @time push!(inf_mus2, infer_mu(tree_file, joinpath(folder, "seq2/seq2_mu$(round(mu, 
               digits = 2)).fa")))
    @time push!(inf_mus3, infer_mu(tree_file, joinpath(folder, "seq3/seq3_mu$(round(mu, 
               digits = 2)).fa")))
end


pp = zeros(21,76)
#analysis on generated data
res_ASR = zeros(Int, 3, L, length(mus)); 
p_ASR = zeros(Int, 3, L, q, length(mus));
for i in 1:3
    inf_mus = readdlm(joinpath(folder, "seq$(i)/inferred_mus$(i).txt"))
    #println(inf_mus)
    for n in 1:length(mus)
        file_seqs = joinpath(folder, "seq$(i)/seq$(i)_mu$(round(mus[n], 
        digits = 2)).fa")
        println(file_seqs)
        res, p = Felsenstein(file_seqs, tree_file, file_seqs, inf_mus[n])
        res_ASR[i,:,n] .= Int.(res)
        p[i,:,:,n] 
    end
end



@load "../data_Anc/DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301.jld2"
step_msa = [step_msa_1, step_msa_2, step_msa_3];
 
#seq1 = [2, 18, 18, 2, 6, 3, 15, 1, 16, 6, 20, 12, 20, 6, 18, 18, 16, 2, 12, 6, 2, 9, 11, 5, 5, 15, 15, 16, 18, 15, 10, 6, 10, 18, 20, 15, 2, 14, 12, 9, 6, 9, 2, 5, 8, 12, 9, 3, 8, 15, 12, 9, 2, 15, 20, 2, 15, 5, 14, 9, 2, 18, 16, 1, 6, 11, 15, 8, 4, 1, 8, 14, 1, 4, 21, 21];
H_ASR = zeros(3,length(mus));        
H_evol = zeros(3,length(mus));
for i in 1:3
    for n in 1:length(mus)
        #H_ASR[i,n] = ham_dist(start_msa[:,i], Int8.(seq1))
        H_ASR[i,n] = ham_dist(Int.(start_msa[:,i]), res_ASR[i,:,n])
        H_evol[i,n] = mean(ham_dist(Int.(start_msa[:,i]), step_msa[i][n]))
    end 
end

cc = ["blue", "red", "green"];
close("all"); 
for i in 1:3
    plt.scatter(sweeps[[1,2,3,4,6]], H_ASR[i,:] ./L, marker = "*", label = "H_asr $(i)", color = cc[i]); 
    plt.scatter(sweeps[[1,2,3,4,6]], H_evol[i,:] ./L, marker = "o", label = "H_evol $(i)", color = cc[i]); 
    plt.xscale("log"); 
    plt.xlabel("Average root-leaf sweeps"); 
    plt.ylabel("Hamming distance");
    plt.legend(); 
end 
savefig("../prova_asr_new.png")


